In [ ]:
import torch
import torch.nn as nn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import arkad
from arkad import BaseModel
from torch.utils.data import Dataset, DataLoader

import sys
import os
from sklearn.metrics.pairwise import cosine_similarity

current_dir = os.getcwd()

sys.path.append(os.path.abspath(os.path.join(current_dir, '..')))

from src.datasets import *
from src.models import *
from src.train_test import *

c:\Users\gabri\anaconda3\envs\nlp_dl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Defina os hiperparâmetros
hidden_dim = [768, 512, 256]
encoding_size = 128
learning_rate = 3e-4
num_epochs = 15
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recrie a instância do modelo (use os mesmos hiperparâmetros)
model_loaded = VAE(
    hidden_dim=hidden_dim,
    encoding_size=encoding_size,
    output_dim_classifier=7,
    dropout=0.5,
    batch_norm_1d=True
)

model_loaded.load_state_dict(torch.load('../models/vae_model_state_dict_2.pth'))
model_loaded = model_loaded.to('cpu')

C:\Users\gabri\AppData\Local\Temp\ipykernel_62644\3310320260.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_loaded.load_state_dict(torch.load('../models/vae_mode

In [3]:
# get csv 
data = pd.read_csv('../data/data.csv')
print(data.shape)

(5414, 3)


In [4]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
tokenized_data = tokenizer(data['text'].tolist(), padding=True, truncation=True, return_tensors='pt')

In [5]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(tokenized_data['input_ids'], tokenized_data['attention_mask'])
dataloader = DataLoader(dataset, batch_size=128)  # Ajuste o batch_size conforme a capacidade da GPU

In [6]:
mu_outputs = []
model_loaded.eval()  # Colocar o modelo em modo de avaliação
model_loaded.to('cuda')  # Mova o modelo para a cuda

with torch.no_grad():  # Desativar o cálculo de gradiente para economizar memória
    for batch in tqdm(dataloader):
        input_ids, attention_mask = batch
        input_ids = input_ids.to('cuda')
        attention_mask = attention_mask.to('cuda')
        outputs = model_loaded.encoder(input_ids=input_ids, attention_mask=attention_mask)
        mu_outputs.append(outputs[0])

# Concatenar os batches para ter o resultado final
mu_outputs = torch.cat(mu_outputs, dim=0)

  0%|          | 0/43 [00:00<?, ?it/s]c:\Users\gabri\anaconda3\envs\nlp_dl\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 43/43 [01:28<00:00,  2.06s/it]


In [7]:
mu_outputs_ = mu_outputs.cpu().detach().numpy()

In [47]:
mu_outputs.shape, data['text'].shape

(torch.Size([5414, 128]), (5414,))

In [8]:
header = ['mu_' + str(i) for i in range(mu_outputs.shape[1])]
np.savetxt('../data/mu_outputs.csv', mu_outputs_, delimiter=',', header=','.join(header), comments='')

In [9]:
# Load data
data_mu = pd.read_csv('../data/mu_outputs.csv')

In [12]:
data_mu.values.shape

(5414, 128)